<a href="https://colab.research.google.com/github/wjm1015/SamsungCard/blob/main/%EB%B0%9C%ED%99%94_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.1"
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re
from pykospacing import Spacing  
from hanspell import spell_checker
from konlpy.tag import Okt

In [4]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/공모전_제공_데이터(1차).xlsx', sheet_name='학습데이터')
rank = pd.read_excel('/content/drive/My Drive/Colab Notebooks/공모전_제공_데이터(1차).xlsx', sheet_name='우선순위')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NO                    1000 non-null   int64  
 1   NO2                   1000 non-null   int64  
 2   발화                    1000 non-null   object 
 3   발화1                   1000 non-null   object 
 4   발화1의 우선순위             1000 non-null   int64  
 5   발화2                   32 non-null     object 
 6   발화2의 우선순위             32 non-null     float64
 7   발화3                   3 non-null      object 
 8   발화3의 우선순위             3 non-null      float64
 9   최종분류(우선순위 가장 높은것 선택)  1000 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 78.2+ KB


In [6]:
rank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   우선순위     44 non-null     int64 
 1   의도명(유형)  44 non-null     object
dtypes: int64(1), object(1)
memory usage: 832.0+ bytes


In [7]:
utterance = df["발화"]

In [8]:
# 한글, 영문, 숫자만 남기고 모두 제거
def preprocessing(text):
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

In [9]:
for i in range(0, len(df)):
    utterance[i] = preprocessing(utterance[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
# 맞춤법 검사
for i in range(len(df)): 
    try:
        result_df = spell_checker.check(utterance[i])
        utterance[i] = result_df.as_dict()['checked']
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
# 띄어쓰기
spacing = Spacing()

for n in range(len(df)):
    utterance[n] = spacing(utterance[n]) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [12]:
utterance[999]

'무이자 혜택이 많아 잘 쓰고 있어요'

In [15]:
# 형태소 분석
for i, document in enumerate(utterance):
    okt = Okt()
    clean_words = []
    for word in okt.pos(document, stem=True): #어간 추출
        if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
            clean_words.append(word[0])
    #print(clean_words) #['상담', '빠르다', '좋다']
    document = ' '.join(clean_words)
   # print(document) #상담 빠르다 좋다
    utterance[i] = document
print(utterance)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0                                  상담 직원 빠르다 연결하다 슴 좋다
1                                     안 기다리다 상담 되어다 좋다
2                                         기다리다 상담 늦어지다
3                                      상담 직원 통화 매우 힘들다
4                                    상담 받다 기다리다 게 불편하다
                            ...                       
995    카드 분실 되다 지다 카드 통해 연락 받다 바로 찾다 잃어버리다 지도 모르다 감사하다
996               개월 무 이자 할부 링크 혜택 다른 카드 구별 되다 기능 편리하다
997                                 무 이자 할부 자주 주다 감사하다
998                        무 이자 혜택 만족 기 할부 만족 자다 쓰다 있다
999                                무 이자 혜택 많다 자다 쓰다 있다
Name: 발화, Length: 1000, dtype: object


In [19]:
type(utterance)

pandas.core.series.Series